# Поиск изображения по запросу

# Прекод

# Сборный проект-4

Вам поручено разработать демонстрационную версию поиска изображений по запросу.

Для демонстрационной версии нужно обучить модель, которая получит векторное представление изображения, векторное представление текста, а на выходе выдаст число от 0 до 1 — покажет, насколько текст и картинка подходят друг другу.

## Описание данных

Данные доступны по [ссылке](https://code.s3.yandex.net/datasets/dsplus_integrated_project_4.zip).

В файле `train_dataset.csv` находится информация, необходимая для обучения: имя файла изображения, идентификатор описания и текст описания. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `train_images` содержатся изображения для тренировки модели.

В файле `CrowdAnnotations.tsv` — данные по соответствию изображения и описания, полученные с помощью краудсорсинга. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.
3. Доля людей, подтвердивших, что описание соответствует изображению.
4. Количество человек, подтвердивших, что описание соответствует изображению.
5. Количество человек, подтвердивших, что описание не соответствует изображению.

В файле `ExpertAnnotations.tsv` содержатся данные по соответствию изображения и описания, полученные в результате опроса экспертов. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.

3, 4, 5 — оценки трёх экспертов.

Эксперты ставят оценки по шкале от 1 до 4, где 1 — изображение и запрос совершенно не соответствуют друг другу, 2 — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует, 3 — запрос и текст соответствуют с точностью до некоторых деталей, 4 — запрос и текст соответствуют полностью.

В файле `test_queries.csv` находится информация, необходимая для тестирования: идентификатор запроса, текст запроса и релевантное изображение. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `test_images` содержатся изображения для тестирования модели.

## Цель работы

Разработать демонстрационную версию поиска изображений по текстовому запросу. Модель должна:
- Получать векторные представления изображения и текста.
- Оценивать их соответствие через число от 0 до 1 (вероятность релевантности).

## План работы

**Исследовательский анализ данных**
- Агрегация экспертных оценок (например, голосование большинства).
- Объединение экспертных и краудсорсинговых оценок (возможно, с весовыми коэффициентами).
- Приведение целевой переменной к диапазону [0, 1].

**Проверка данных**

- Удаление изображений, нарушающих законодательство (например, с участием детей до 16 лет).

**Векторизация изображений**
- Использование предобученной CNN (например, ResNet-18) для извлечения признаков (исключая полносвязные слои).

**Векторизация текстов**
- Эксперименты с методами:
    - TF-IDF
    - Word2Vec
    - Трансформеры (например, BERT).

**Объединение векторов**
- Создание общего датасета с объединёнными векторами изображений, текстов и целевой переменной.

**Обучение модели предсказания соответствия**
- Разделение данных с учётом уникальности изображений (GroupShuffleSplit).
- Выбор модели (например, логистическая регрессия, градиентный бустинг или нейросеть).
- Определение метрики качества (например, ROC-AUC или кастомная метрика).

**Тестирование модели**
- Оценка качества на тестовой выборке.
- Проверка работы на данных из test_queries.csv и test_images.

## Ожидаемый результат

- Модель, предсказывающая вероятность соответствия текста и изображения (0–1).
- Метрика качества, отражающая точность ранжирования пар (например, ROC-AUC).
- Готовый пайплайн от обработки данных до предсказания для демонстрации поиска.

**Дополнительные требования:**
- Юридическая проверка изображений.
- Учёт уникальности изображений при разбиении на выборки.
- Гибкость в выборе методов векторизации и моделей.

## Исследовательский анализ данных

Наш датасет содержит экспертные и краудсорсинговые оценки соответствия текста и изображения.

В файле с экспертными мнениями для каждой пары изображение-текст имеются оценки от трёх специалистов. Для решения задачи вы должны эти оценки агрегировать — превратить в одну. Существует несколько способов агрегации оценок, самый простой — голосование большинства: за какую оценку проголосовала большая часть экспертов (в нашем случае 2 или 3), та оценка и ставится как итоговая. Поскольку число экспертов меньше числа классов, может случиться, что каждый эксперт поставит разные оценки, например: 1, 4, 2. В таком случае данную пару изображение-текст можно исключить из датасета.

Вы можете воспользоваться другим методом агрегации оценок или придумать свой.

В файле с краудсорсинговыми оценками информация расположена в таком порядке:

1. Доля исполнителей, подтвердивших, что текст **соответствует** картинке.
2. Количество исполнителей, подтвердивших, что текст **соответствует** картинке.
3. Количество исполнителей, подтвердивших, что текст **не соответствует** картинке.

После анализа экспертных и краудсорсинговых оценок выберите либо одну из них, либо объедините их в одну по какому-то критерию: например, оценка эксперта принимается с коэффициентом 0.6, а крауда — с коэффициентом 0.4.

Ваша модель должна возвращать на выходе вероятность соответствия изображения тексту, поэтому целевая переменная должна иметь значения от 0 до 1.


### Установка необходимых библиотек

In [1]:
import os
from typing import Generator
from pathlib import Path
from PIL import Image
import re

import pandas as pd
import numpy as np
import nltk
import cv2
from deepface import DeepFace
from mtcnn import MTCNN
import gensim.downloader as api

import torch
import torchvision.models as models
from torchvision import transforms

2025-08-06 00:03:43.232771: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-06 00:03:43.245360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754431423.256898  165675 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754431423.260276  165675 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754431423.269550  165675 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Загрузка необходимых ресурсов NLTK
nltk.data.path.append('datasets/nltk_data')

### Вспомогательные функции

In [3]:
# Инициализация детектора лиц (MTCNN)
detector = MTCNN()


def has_child(image_dir: Path, img_names: pd.Series,  age_threshold: int = 16) -> Generator[str, None, None]:
    """
    Проверяет, есть ли на изображении дети младше `age_threshold`.
    Возвращает наименование файла изображения на котором есть дети младше `age_threshold`.
    """

    for img_name in img_names:
        image_path = Path(image_dir) / img_name
        try:
            img = cv2.cvtColor(cv2.imread(image_path.as_posix()), cv2.COLOR_BGR2RGB)
            faces = detector.detect_faces(img)  # Детекция лиц

            for face in faces:
                x, y, w, h = face["box"]
                face_img = img[y:y + h, x:x + w]  # Обрезаем лицо

                # Оценка возраста
                analysis = DeepFace.analyze(face_img, actions=["age"], enforce_detection=False)
                if analysis[0]["age"] < age_threshold:
                    yield image_path.name

        except Exception as ex:
            print(f"Ошибка при обработке {image_path}: {ex}")


# Инициализация лемматизатора и стоп-слов
lemmatizer = nltk.stem.WordNetLemmatizer()

def get_wordnet_pos(treebank_tag: str):
    """
    Конвертирует POS-теги Penn Treebank в формат WordNet
    """

    tag = treebank_tag[0].upper()
    tag_dict = {
        "J": nltk.corpus.wordnet.ADJ,
        "N": nltk.corpus.wordnet.NOUN,
        "V": nltk.corpus.wordnet.VERB,
        "R": nltk.corpus.wordnet.ADV
    }
    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)

def get_lemmas(text: str, ngram: int = 1) -> set[tuple]:
    """
    Лемматизация текста и проверка слов исключений
    :param text: исходный текст
    :param ngram: количество лемм в n-грамме
    """
    
    # Токенизация и POS-тегирование
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z\s]", " ", text.lower()))
    
    # Генерируем n-граммы
    pos_tags = nltk.tag.pos_tag(tokens)
    
    # Лемматизация с учетом POS
    lemmas: list[str] = [
        lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag))
        for word, tag in pos_tags
    ]
    return set(nltk.ngrams(lemmas, ngram))

2025-08-06 00:03:47.327989: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### Загрузка данных

In [4]:
# загружаем данные по соответствию изображения и описания, полученные в результате опроса экспертов
expert_df: pd.DataFrame = pd.read_csv(
    'datasets/ExpertAnnotations.tsv', 
    sep='\t', 
    header=None, 
    names=['image', 'desc_id', 'expert1', 'expert2', 'expert3'],
)

# загружаем данные по соответствию изображения и описания, полученные с помощью краудсорсинга
сrowd_df: pd.DataFrame = pd.read_csv(
    'datasets/CrowdAnnotations.tsv', 
    sep='\t', 
    header=None, 
    names=['image', 'desc_id', 'match_ratio', 'match_count', 'non_match_count'],
)

# загружаем информацию необходимую для обучения
train_dataset_df: pd.DataFrame = pd.read_csv('datasets/train_dataset.csv')

# загружаем информацию необходимую для тестирования
test_queries_df: pd.DataFrame = pd.read_csv('datasets/test_queries.csv', sep='|',)

In [5]:
# проверим состав данных и соответствие типов для ExpertAnnotations.tsv
expert_df.info()
expert_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    5822 non-null   object
 1   desc_id  5822 non-null   object
 2   expert1  5822 non-null   int64 
 3   expert2  5822 non-null   int64 
 4   expert3  5822 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 227.5+ KB


,image,desc_id,expert1,expert2,expert3
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2


In [6]:
# проверим состав данных и соответствие типов для CrowdAnnotations.tsv
сrowd_df.info()
сrowd_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47830 entries, 0 to 47829
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   image            47830 non-null  object 
 1   desc_id          47830 non-null  object 
 2   match_ratio      47830 non-null  float64
 3   match_count      47830 non-null  int64  
 4   non_match_count  47830 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ MB


,image,desc_id,match_ratio,match_count,non_match_count
0,1056338697_4f7d7ce270.jpg,1056338697_4f7d7ce270.jpg#2,1.0,3,0
1,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.0,0,3
2,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.0,0,3
3,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.0,0,3
4,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.0,0,3


In [7]:
# проверим состав данных и соответствие типов для train_dataset.csv
train_dataset_df.info()
train_dataset_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB


,image,query_id,query_text
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,1262583859_653f1469a9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2447284966_d6bbdb4b6e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2621415349_ef1a7e73be.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...


In [8]:
# проверим состав данных и соответствие типов для test_queries.csv
test_queries_df.info()
test_queries_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  500 non-null    int64 
 1   query_id    500 non-null    object
 2   query_text  500 non-null    object
 3   image       500 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB


,Unnamed: 0,query_id,query_text,image
0,0,1177994172_10d143cb8d.jpg#0,"Two blonde boys , one in a camouflage shirt an...",1177994172_10d143cb8d.jpg
1,1,1177994172_10d143cb8d.jpg#1,Two boys are squirting water guns at each other .,1177994172_10d143cb8d.jpg
2,2,1177994172_10d143cb8d.jpg#2,Two boys spraying each other with water,1177994172_10d143cb8d.jpg
3,3,1177994172_10d143cb8d.jpg#3,Two children wearing jeans squirt water at eac...,1177994172_10d143cb8d.jpg
4,4,1177994172_10d143cb8d.jpg#4,Two young boys are squirting water at each oth...,1177994172_10d143cb8d.jpg


###  Предобработка данных

В данных датасета `train_dataset_df` данные в поле `image` не соответствуют `query_id`, исправим это.

In [9]:
train_dataset_df["image"] = train_dataset_df["query_id"].apply(lambda x: x.split("#")[0])
train_dataset_df

,image,query_id,query_text
0,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
...,...,...,...
5817,2061144717_5b3a1864f0.jpg,2061144717_5b3a1864f0.jpg#2,A man in an ampitheater talking to a boy .
5818,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .
5819,2112921744_92bf706805.jpg,2112921744_92bf706805.jpg#2,A dog stands on the side of a grassy cliff .
5820,3651971126_309e6a5e22.jpg,3651971126_309e6a5e22.jpg#2,A blurry photo of two dogs .


In [10]:
# избавимся от лишнего поля
test_queries_df = test_queries_df.drop(["Unnamed: 0"], axis=1)
test_queries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   query_id    500 non-null    object
 1   query_text  500 non-null    object
 2   image       500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


Проверим данные на наличие дубликатов

In [11]:
print("expert_df:", expert_df.duplicated().sum())
print("сrowd_df:", сrowd_df.duplicated().sum())
print("train_dataset_df:", train_dataset_df.duplicated().sum())

expert_df: 0
сrowd_df: 0
train_dataset_df: 4845


**Вывод:**
- исправлено ошибочно заполненное поле `image` датасета `test_queries_df`
- удалено лишнее поля из датафрейма `test_queries_df`
- данные проверены на наличие дубликатов

### Поиск финального скоринга как вероятность соответствия описания картинке

#### Обработка краудсорсинговых данных
- Долю подтверждений (`match_ratio`).
- Количество подтверждений (`match_count`).
- Количество опровержений (`non_match_count`).

Можно использовать долю подтверждений (`match_ratio`) как вероятность соответствия.

#### Агрегация по большинству голосов для экспертных данных

In [12]:
# Агрегация по большинству голосов
expert_df["expert_vote"] = (expert_df[["expert1", "expert2", "expert3"]].sum(axis=1) >= 2).astype(int)

In [13]:
# Удаление строк без консенсуса (все эксперты дали разные оценки)
expert_df = expert_df[expert_df[["expert1", "expert2", "expert3"]].nunique(axis=1) != 3]

In [14]:
expert_df.head()

,image,desc_id,expert1,expert2,expert3,expert_vote
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1,1
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2,1
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2,1
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2,1
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2,1


#### Объединение экспертных и краудсорсинговых оценок
Есть несколько вариантов:
- Использовать только экспертные оценки (более надежные, но данных меньше).
- Использовать только краудсорсинговые оценки (больше данных, но возможен шум).
- Комбинировать оценки, например:
    - `final_score = 0.6 * expert_vote + 0.4 * match_ratio`

В данном случае воспользуемся вариантом `комбинировать оценки`

In [15]:
merged_df = pd.merge(expert_df, сrowd_df, on=["image", "desc_id"], how="inner")
merged_df["final_score"] = round(0.6 * merged_df["expert_vote"] + 0.4 * merged_df["match_ratio"], 2)

In [16]:
merged_df

,image,desc_id,expert1,expert2,expert3,expert_vote,match_ratio,match_count,non_match_count,final_score
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1,1,0.000000,0,3,0.60
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2,1,0.000000,0,3,0.60
2,1056338697_4f7d7ce270.jpg,434792818_56375e203f.jpg#2,1,1,2,1,0.000000,0,3,0.60
3,1084040636_97d9633581.jpg,256085101_2c2617c5d0.jpg#2,2,3,3,1,0.333333,1,2,0.73
4,1084040636_97d9633581.jpg,3396157719_6807d52a81.jpg#2,1,2,2,1,0.000000,0,3,0.60
...,...,...,...,...,...,...,...,...,...,...
2253,979383193_0a542a059d.jpg,3244747165_17028936e0.jpg#2,2,2,2,1,0.000000,0,3,0.60
2254,979383193_0a542a059d.jpg,3482062809_3b694322c4.jpg#2,1,2,2,1,0.000000,0,3,0.60
2255,997722733_0cb5439472.jpg,2985679744_75a7102aab.jpg#2,1,1,1,1,0.000000,0,3,0.60
2256,997722733_0cb5439472.jpg,3150742439_b8a352e1e0.jpg#2,1,1,2,1,0.000000,0,3,0.60


**Вывод:**
- была выполнена агрегация по большинству голосов для экспертных данных
- объединенили экспертные и краудсорсинговые оценок, в результате чего получили итоговый скоринг `final_score`.

## 2. Проверка данных

В некоторых странах, где работает ваша компания, действуют ограничения по обработке изображений: поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно тексты, изображения, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16 лет.

В вашем сервисе строго следуют законам стран, в которых работают. Поэтому при попытке посмотреть изображения, запрещённые законодательством, вместо картинок показывается дисклеймер:

> This image is unavailable in your country in compliance with local laws
>

Однако у вас в PoC нет возможности воспользоваться данным функционалом. Поэтому все изображения, которые нарушают данный закон, нужно удалить из обучающей выборки.

Поскольку в вашем PoC нет автоматического определения запрещённого контента, потребуется ручная или полуавтоматическая очистка данных.

**Определение критериев для удаления:**
- В метаданных/описании необходимо найти слова которые однозначно указывают на детей и на их основе исключить изображения. Данные следует подкрепить оценкой соответствия текста и изображения для повышения точности.
- Есть лица людей, похожих на детей (возраст < 16 лет).

Основные леммы, связанные с детьми (до 16 лет)

**1. Общие термины**
- child (ребёнок)
- kid (ребёнок, разг.)
- baby (младенец, малыш)
- toddler (ребёнок 1-3 лет)
- infant (младенец, до 1 года)
- minor (несовершеннолетний)
- juvenile (несовершеннолетний, юный)
- youngster (ребёнок, подросток)

**2. Возрастные группы**
- preschooler (дошкольник, 3-5 лет)
- schoolchild / schoolkid (школьник)
- preteen (ребёнок 9-12 лет)
- pupil (ученик начальной школы)

**3. Другие связанные термины**
- underage (несовершеннолетний)
- young + boy/girl (мальчик/девочка)
- little + kid/child/boy/girl (маленький ребёнок)

In [17]:
# попробуем сначало найти ключевые биграммы
unique_bigrams: set[str] = set()
for text in train_dataset_df["query_text"].unique():
    unique_bigrams.update(get_lemmas(text, 2))

In [18]:
for row in sorted(unique_bigrams, key=lambda x: x[0]):
    if {
        "underage", 
        "young",
        "little",
        "small",       
    } & set(row):
        print(row)

('a', 'little')
('a', 'small')
('a', 'young')
('and', 'young')
('five', 'young')
('four', 'small')
('little', 'dog')
('little', 'girls')
('little', 'drum')
('little', 'kid')
('little', 'child')
('little', 'boy')
('little', 'brown')
('little', 'girl')
('own', 'little')
('small', 'plane')
('small', 'girl')
('small', 'black')
('small', 'poster')
('small', 'dog')
('small', 'crowd')
('small', 'waterfall')
('small', 'object')
('small', 'white')
('small', 'kangaroo')
('small', 'tree')
('small', 'child')
('small', 'boy')
('small', 'animal')
('small', 'ball')
('small', 'one')
('small', 'boat')
('small', 'art')
('small', 'river')
('small', 'brown')
('smile', 'little')
('the', 'young')
('the', 'little')
('the', 'small')
('three', 'young')
('three', 'little')
('three', 'small')
('two', 'little')
('two', 'small')
('while', 'young')
('with', 'small')
('young', 'man')
('young', 'long')
('young', 'football')
('young', 'woman')
('young', 'person')
('young', 'adult')
('young', 'boy')
('young', 'blond')


Из полученного списка можно выделить несколько биграмм определяющих ребенка на изображении.

In [19]:
child_phrases: set[tuple[str, str]] = {
    ('little', 'boy'),
    ('little', 'kid'),
    ('little', 'child'),
    ('little', 'girl'),
    ('little', 'girls'),
    ('small', 'girl'),
    ('small', 'boy'),
    ('small', 'child'),
    ('young', 'boy'),
    ('young', 'child'),
    ('young', 'girl'),
}

Выполним поик по ключевым словам

In [20]:
# попробуем сначало найти ключевые биграммы
unique_bigrams: set[str] = set()
for text in train_dataset_df["query_text"].unique():
    unique_bigrams.update(get_lemmas(text, 1))

In [21]:
for row in sorted(unique_bigrams, key=lambda x: x[0]):
    if {
        "child",
        "kid",
        "baby",
        "toddler", 
        "infant", 
        "minor", 
        "juvenile", 
        "youngster",
        "preschooler",
        "schoolchild",
        "schoolkid",
        "preteen",
        "pupil",
    } & set(row):
        print(row)

('baby',)
('child',)
('kid',)
('toddler',)


В результате получился следующий список:

In [22]:
child_words: set[tuple[str]] = {
    ('baby',),
    ('child',),
    ('kid',),
    ('toddler',),
}

In [23]:
def check_exception_words(text: str) -> int:
    lammas_1: set[tuple[str]] = get_lemmas(text, 1)
    lammas_2: set[tuple[str, str]] = get_lemmas(text, 2)
    if child_words & lammas_1:
        return 1
    if child_phrases & lammas_2:
        return 1
    return 0

In [24]:
# найдем слова исключения в тренировочных данных
train_dataset_df["has_child"] = train_dataset_df["query_text"].apply(check_exception_words)

In [25]:
# объеденим данные с оценками соответствия и тренировочную выборку
merged_train_dataset_df = pd.merge(train_dataset_df, merged_df[["image", "final_score"]], on=["image"], how="inner")
merged_train_dataset_df

,image,query_id,query_text,has_child,final_score
0,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,1,0.60
1,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,1,1.00
2,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,1,0.60
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,1,1.00
4,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,1,0.60
...,...,...,...,...,...
13895,2112921744_92bf706805.jpg,2112921744_92bf706805.jpg#2,A dog stands on the side of a grassy cliff .,0,0.60
13896,2112921744_92bf706805.jpg,2112921744_92bf706805.jpg#2,A dog stands on the side of a grassy cliff .,0,0.60
13897,3651971126_309e6a5e22.jpg,3651971126_309e6a5e22.jpg#2,A blurry photo of two dogs .,0,0.73
13898,2292406847_f366350600.jpg,2292406847_f366350600.jpg#2,A man rows his boat below .,0,0.60


Отберем те записи у которых `final_score > 0.8` и `has_children = 0`. Значит, что картинка полностью соответствует описанию и в описании есть дети.

In [26]:
imgs_without_children = merged_train_dataset_df[
    (merged_train_dataset_df["has_child"] == 0) & (merged_train_dataset_df["final_score"] > 0.7)
]
imgs_without_children

,image,query_id,query_text,has_child,final_score
24,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00
27,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00
30,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00
33,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00
36,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00
...,...,...,...,...,...
13866,523985664_c866af4850.jpg,523985664_c866af4850.jpg#2,A girl wearing a swimsuit top wears a sign to ...,0,0.73
13881,3200120942_59cfbb3437.jpg,3200120942_59cfbb3437.jpg#2,One person is on a ski lift while another snow...,0,0.87
13891,2061144717_5b3a1864f0.jpg,2061144717_5b3a1864f0.jpg#2,A man in an ampitheater talking to a boy .,0,0.73
13894,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,0,0.87


Попробуем найти лица людей, похожих на детей (возраст < 16 лет).

In [27]:
train_image_dir: list[str] = ["datasets", "train_images"]

In [28]:
exclude_img: set[str] = set() 
for img_name in has_child(image_dir=Path(os.getcwd(), *train_image_dir), img_names=imgs_without_children["image"].unique()):
    exclude_img.add(img_name)

2025-08-06 00:03:55.356293: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Must provide as many biases as the last dimension of the input tensor: [32] vs. [0,48,48,3]
2025-08-06 00:03:56.202958: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Must provide as many biases as the last dimension of the input tensor: [32] vs. [0,48,48,3]
2025-08-06 00:03:56.385941: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Must provide as many biases as the last dimension of the input tensor: [32] vs. [0,48,48,3]
2025-08-06 00:04:04.029436: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Must provide as many biases as the last dimension of the input tensor: [32] vs. [0,48,48,3]
2025-08-06 00:04:07.516420: I tensorflow/core/framework/local_rendez

In [29]:
# проверим наличие 
exclude_img

set()

**Вывод:**
- для поиска детей до 16 лет использовал:
    - поиск по ключевым словам и оценку соответствия текста и изображения.
    - поиск по изображению лиц младше 16 лет.

## 3. Векторизация изображений

Перейдём к векторизации изображений.

Самый примитивный способ — прочесть изображение и превратить полученную матрицу в вектор. Такой способ нам не подходит: длина векторов может быть сильно разной, так как размеры изображений разные. Поэтому стоит обратиться к свёрточным сетям: они позволяют "выделить" главные компоненты изображений. Как это сделать? Нужно выбрать какую-либо архитектуру, например ResNet-18, посмотреть на слои и исключить полносвязные слои, которые отвечают за конечное предсказание. При этом можно загрузить модель данной архитектуры, предварительно натренированную на датасете ImageNet.

In [30]:
# Загружаем предобученную ResNet18
resnet = models.resnet18(pretrained=True)

# Удаляем последний полносвязный слой (avgpool и fc)
resnet = torch.nn.Sequential(*(list(resnet.children())[:-2]))  # Оставляем слои до avgpool

# Стандартные трансформации для ImageNet
preprocess = transforms.Compose([
    transforms.Resize(256),          # Изменение размера с сохранением пропорций
    transforms.CenterCrop(224),      # Центральное обрезание до 224x224
    transforms.ToTensor(),           # Преобразование в тензор
    transforms.Normalize(            # Нормализация
        mean=[0.485, 0.456, 0.406],  # Средние значения для ImageNet
        std=[0.229, 0.224, 0.225]   # Стандартные отклонения для ImageNet
    )
])

In [31]:
def extract_features(image_name: str) -> np.ndarray:
    """
    Извлекает признаки (feature vector) из изображения с помощью предобученной модели.

    Параметры:
        image_name: Имя файла
        
    Возвращает:
        1D массив numpy с извлеченными признаками

    Описание работы:
        1. Загружает изображение по указанному пути
        2. Применяет препроцессинг (предполагается, что функция preprocess() определена где-то)
        3. Добавляет размерность батча для обработки моделью
        4. Пропускает изображение через модель без вычисления градиентов
        5. Усредняет признаки по пространственным измерениям (H и W)
        6. Преобразует результат в 1D numpy массив
    """

    # Загрузка и препроцессинг изображения
    img = Image.open(Path(*train_image_dir, image_name))
    img_t = preprocess(img)
    batch_t = torch.unsqueeze(img_t, 0)  # Добавляем размерность батча

    # Извлечение признаков
    with torch.no_grad():  # Отключаем вычисление градиентов
        features = resnet(batch_t)

    # Преобразуем 4D-тензор в 1D-вектор (усреднение по пространственным измерениям)
    features = torch.mean(features, dim=[2, 3]).squeeze()

    return features.numpy()

In [32]:
# Переводим к векторам изображения
imgs_without_children["vector"] = imgs_without_children["image"].apply(extract_features)
imgs_without_children

,image,query_id,query_text,has_child,final_score,vector
24,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ..."
27,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ..."
30,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ..."
33,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ..."
36,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ..."
...,...,...,...,...,...,...
13866,523985664_c866af4850.jpg,523985664_c866af4850.jpg#2,A girl wearing a swimsuit top wears a sign to ...,0,0.73,"[0.9201368, 0.99850404, 0.92134064, 0.9980819,..."
13881,3200120942_59cfbb3437.jpg,3200120942_59cfbb3437.jpg#2,One person is on a ski lift while another snow...,0,0.87,"[0.8865715, 0.92025, 0.87535477, 0.94944173, 0..."
13891,2061144717_5b3a1864f0.jpg,2061144717_5b3a1864f0.jpg#2,A man in an ampitheater talking to a boy .,0,0.73,"[0.91834086, 1.0169836, 0.91914463, 1.0726757,..."
13894,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,0,0.87,"[0.87389714, 1.0444498, 1.0732609, 1.1024729, ..."


**Вывод:**
- для векторизации изображений использовали модель `ResNet18`
- векторизацию применили к датасету с ранее отобранными изображениями (без детей младше 16 лет)

## 4. Векторизация текстов

Следующий этап — векторизация текстов. Вы можете поэкспериментировать с несколькими способами векторизации текстов:

- tf-idf
- word2vec
- \*трансформеры (например Bert)

\* — если вы изучали трансформеры в спринте Машинное обучение для текстов.


Для векторизации текста будем использовать `word2vec` модель.

**Плюсы**:
- Учитывает семантику слов
- Фиксированный размер вектора (300 в примере)
- Улавливает отношения между словами
- 
**Минусы**:
- Требует обучения (или загрузки предобученной модели)
- Потеря порядка слов (если просто усреднять)
- Не учитывает полный контекст предложения

Для баланса качества и скорости - `Word2Vec` (загрузим предобученную модель)

In [33]:
# Загрузка предобученной модели (300-мерные векторы)
w2v_model = api.load('word2vec-google-news-300')

In [34]:
def document_vector_pretrained(text: str) -> np.ndarray:
    """
    Преобразует текст в векторное представление (документный эмбеддинг) 
    с использованием предобученной word2vec модели.
    
    Для каждого слова в тексте извлекается вектор из word2vec модели,
    затем возвращается среднее всех векторов слов (усредненный эмбеддинг документа).
    
    Параметры:
        text: Входной текст для векторного представления
        
    Возвращает:
        Векторное представление документа (усредненный вектор слов).
        Если ни одно слово не найдено в модели, возвращается нулевой вектор
        размерности w2v_model.vector_size.
    """
    words = nltk.tokenize.word_tokenize(text.lower())
    word_vectors = [w2v_model[word] for word in words if word in w2v_model]
    if len(word_vectors) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(word_vectors, axis=0)

In [42]:
train_dataset_without_children_df = pd.merge(
    merged_train_dataset_df[["image", "query_id", "query_text", "final_score"]], 
    imgs_without_children[["image", "vector"]], 
    on=["image"], 
    how="inner",
)
train_dataset_without_children_df["text_vector"] = train_dataset_without_children_df["query_text"].apply(document_vector_pretrained)

In [43]:
train_dataset_without_children_df

,image,query_id,query_text,final_score,vector,text_vector
0,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ...","[0.0072631836, 0.058972675, 0.06608073, 0.0617..."
1,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ...","[0.0072631836, 0.058972675, 0.06608073, 0.0617..."
2,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ...","[0.0072631836, 0.058972675, 0.06608073, 0.0617..."
3,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ...","[0.0072631836, 0.058972675, 0.06608073, 0.0617..."
4,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ...","[0.0072631836, 0.058972675, 0.06608073, 0.0617..."
...,...,...,...,...,...,...
141255,2061144717_5b3a1864f0.jpg,2061144717_5b3a1864f0.jpg#2,A man in an ampitheater talking to a boy .,0.73,"[0.91834086, 1.0169836, 0.91914463, 1.0726757,...","[0.17399089, 0.13090007, 0.098225914, -0.00506..."
141256,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,0.60,"[0.87389714, 1.0444498, 1.0732609, 1.1024729, ...","[0.12083217, 0.105922155, 0.044154577, 0.05060..."
141257,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,0.60,"[0.87389714, 1.0444498, 1.0732609, 1.1024729, ...","[0.12083217, 0.105922155, 0.044154577, 0.05060..."
141258,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,0.87,"[0.87389714, 1.0444498, 1.0732609, 1.1024729, ...","[0.12083217, 0.105922155, 0.044154577, 0.05060..."


Удалим дубликаты в данных

In [46]:
train_dataset_without_children_df = train_dataset_without_children_df.drop_duplicates(subset=["image", "query_id", "query_text"])

In [45]:
train_dataset_without_children_df

,image,query_id,query_text,final_score,vector,text_vector
0,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,1.00,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ...","[0.0072631836, 0.058972675, 0.06608073, 0.0617..."
10,2718495608_d8533e3ac5.jpg,2718495608_d8533e3ac5.jpg#2,A girl wearing a yellow shirt and sunglasses s...,0.60,"[0.94001395, 0.9511131, 0.9383936, 1.0881505, ...","[0.0072631836, 0.058972675, 0.06608073, 0.0617..."
300,3286822339_5535af6b93.jpg,3286822339_5535af6b93.jpg#2,Chinese market street in the winter time .,0.60,"[0.862807, 0.97104704, 0.87188107, 1.0272604, ...","[-0.035609655, 0.101850234, 0.016313825, 0.076..."
307,3286822339_5535af6b93.jpg,3286822339_5535af6b93.jpg#2,Chinese market street in the winter time .,0.87,"[0.862807, 0.97104704, 0.87188107, 1.0272604, ...","[-0.035609655, 0.101850234, 0.016313825, 0.076..."
398,1425069308_488e5fcf9d.jpg,1425069308_488e5fcf9d.jpg#2,A white dog jumps to catch a blue ball in a li...,1.00,"[0.9268372, 0.97070444, 0.96834815, 1.0649868,...","[0.052341036, 0.06041124, -0.00032552084, 0.05..."
...,...,...,...,...,...,...
141254,2061144717_5b3a1864f0.jpg,2061144717_5b3a1864f0.jpg#2,A man in an ampitheater talking to a boy .,0.60,"[0.91834086, 1.0169836, 0.91914463, 1.0726757,...","[0.17399089, 0.13090007, 0.098225914, -0.00506..."
141255,2061144717_5b3a1864f0.jpg,2061144717_5b3a1864f0.jpg#2,A man in an ampitheater talking to a boy .,0.73,"[0.91834086, 1.0169836, 0.91914463, 1.0726757,...","[0.17399089, 0.13090007, 0.098225914, -0.00506..."
141256,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,0.60,"[0.87389714, 1.0444498, 1.0732609, 1.1024729, ...","[0.12083217, 0.105922155, 0.044154577, 0.05060..."
141258,2196107384_361d73a170.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,0.87,"[0.87389714, 1.0444498, 1.0732609, 1.1024729, ...","[0.12083217, 0.105922155, 0.044154577, 0.05060..."


## 5. Объединение векторов

Подготовьте данные для обучения: объедините векторы изображений и векторы текстов с целевой переменной.

## 6. Обучение модели предсказания соответствия

Для обучения разделите датасет на тренировочную и тестовую выборки. Простое случайное разбиение не подходит: нужно исключить попадание изображения и в обучающую, и в тестовую выборки.
Для того чтобы учесть изображения при разбиении, можно воспользоваться классом [GroupShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) из библиотеки sklearn.model_selection.

Код ниже разбивает датасет на тренировочную и тестовую выборки в пропорции 7:3 так, что строки с одинаковым значением 'group_column' будут содержаться либо в тестовом, либо в тренировочном датасете.

```
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indices, test_indices = next(gss.split(X=df.drop(columns=['target']), y=df['target'], groups=df['group_column']))
train_df, test_df = df.loc[train_indices], df.loc[test_indices]

```

Какую модель использовать — выберите самостоятельно. Также вам предстоит выбрать метрику качества либо реализовать свою.

## 7. Тестирование модели

Настало время протестировать модель. Для этого получите эмбеддинги для всех тестовых изображений из папки `test_images`, выберите случайные 10 запросов из файла `test_queries.csv` и для каждого запроса выведите наиболее релевантное изображение. Сравните визуально качество поиска.

## 8. Выводы

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Исследовательский анализ данных выполнен
- [ ]  Проверены экспертные оценки и краудсорсинговые оценки
- [ ]  Из датасета исключены те объекты, которые выходят за рамки юридических ограничений
- [ ]  Изображения векторизованы
- [ ]  Текстовые запросы векторизованы
- [ ]  Данные корректно разбиты на тренировочную и тестовую выборки
- [ ]  Предложена метрика качества работы модели
- [ ]  Предложена модель схожести изображений и текстового запроса
- [ ]  Модель обучена
- [ ]  По итогам обучения модели сделаны выводы
- [ ]  Проведено тестирование работы модели
- [ ]  По итогам тестирования визуально сравнили качество поиска